In [1]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [2]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT * 
FROM Z3DMC.MEB_DGM
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
#Po wykonaniu LEFT JOIN liczba rekordów się zwiększyła, trzeba usunąć duplikaty z DMC_CASTING ~ LO
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY')
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
SELECT ID_DMC, CIRCUIT_NR, 
MAX(ASSIGMENT) AS ASSIGMENT, 
MAX(FLOW) AS FLOW, 
MAX(REF_TIME) AS REF_TIME,
MAX(SET_POINT) AS SET_POINT,
MAX(START_DELAY) AS START_DELAY,
MAX(TEMP) AS TEMP,
MAX(WORKING_MODE) AS WORKING_MODE,
MAX(TIMESTAMP) AS TIMESTAMP
FROM Z3DMC.ONI_CIRCUITS
GROUP BY ID_DMC, CIRCUIT_NR
ORDER BY ID_DMC
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [23]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [24]:
print(list(data.keys()))
# Useless tables
# MEB_CONTAINERS, MEB_DMC_GRADE, MEB_KS_WZORC

['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']


In [25]:
# ONI CIRCUITS
# print(data['ONI_CIRCUITS'].head())
data['ONI_CIRCUITS'].drop(columns=['timestamp'], inplace=True)
# print("\n ONI_CIRCUITS after drop of columns: \n")
# print(data['ONI_CIRCUITS'].head())

In [26]:
# MEB_DGM
# print(data['MEB_DGM'].columns)
# print(data['MEB_DGM'].head(5))
# ('metal_level', 'metal_pressure') - We do not collect this data yet
data['MEB_DGM'].drop(columns=['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'], inplace=True)
# print("\n MEB_DGM after drop of columns: \n")
# print(data['MEB_DGM'].head(5))

In [27]:
#MEB_DMC
# print(data['MEB_DMC'].columns)
# print(data['MEB_DMC'].head(5))
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step'], inplace = True)
# print("\n MEB_DMC after drop of columns: \n")
# print(data['MEB_DMC'].head(5))

In [28]:
#MEB_KO
# print(data['MEB_KO'].columns)
# print(data['MEB_KO'].head(5))
data['MEB_KO'].drop(columns = ['id_ko', 'data', 'timestamp', 'eks'], inplace = True)
# print("\n MEB_KO after drop of columns: \n")
# print(data['MEB_KO'].head(5))

In [29]:
#MEB_KO_DGM
# print(data['MEB_KO_DGM'].columns)
# print(data['MEB_KO_DGM'].head(5))
data['MEB_KO_DGM'].drop(columns = ['id_ko','data_odlania', 'timestamp', 'operator'], inplace = True)
# print("\n MEB_KO_DGM after drop of columns: \n")
# print(data['MEB_KO_DGM'].head(5))

In [30]:
# MEB_KS
# print(data['MEB_KS'].columns)
# print(data['MEB_KS'].head(5))
data['MEB_KS'].drop(columns = ['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny'], inplace = True)
# print("\n 'MEB_KS' after drop of columns: \n")
# print(data['MEB_KS'].head(5))

In [31]:
#MEB_GROB 
# print(data['MEB_GROB'].columns)
# print(data['MEB_GROB'].head(5))
data['MEB_GROB'].drop(columns = ['id_meb_grob', 'shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested', 
                                'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'], inplace = True)
# print("\n 'MEB_GROB' after drop of columns: \n")
# print(data['MEB_GROB'].head(5))

<h1>Łączenie tabel</h1>

In [32]:
# usuwanie znaków białych z DMC i DMC_CASTING
data['MEB_DMC'].dmc_casting = data['MEB_DMC'].dmc_casting.str.strip()
data['MEB_DGM'].dmc = data['MEB_DGM'].dmc.str.strip()

In [33]:
# usuwanie z meb_dmc wierszy z 'WORKPIECE NIO' w kodzie DMC
# print(data['MEB_DMC'].shape)
data['MEB_DMC'] = data['MEB_DMC'][~data['MEB_DMC']['dmc'].str.contains('WORKPIECE', case=False, na=False)]
# print(data['MEB_DMC'].shape)

In [41]:
# wybieranie rekordów dla MEB+ 
data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM'].nr_dgm.between(8, 10)) & (data['MEB_DGM'].dmc.apply(lambda x: len(x)) == 21)]

In [42]:
# usunięcie anomalii z MEB_DMC
final_table = data['MEB_DMC'].copy()
final_table = final_table[final_table.dmc.str[:3] == '0MH']

In [44]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)


#data['MEB_KO'].head()
#data['MEB_KO_DGM'].head()

In [45]:
# łączę tabelę MEB_DMC z tabelą MEB_KO
# print(final_table.shape)
final_table = final_table.merge(data['MEB_KO'], on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
# print(final_table.shape)

In [46]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB
# print(final_table.shape)
final_table = final_table.merge(data['MEB_GROB'], on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
# print(final_table.shape)

In [47]:
# łączę tabelę MEB_DMC z tabelą MEB_KS
# print(final_table.shape)
final_table = final_table.merge(data['MEB_KS'], on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
# print(final_table.shape)

In [48]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
# print(data['MEB_DGM'].shape)
data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
data['MEB_DGM'].drop(columns=['rn'], inplace=True)
# print(data['MEB_DGM'].shape)

In [49]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 

oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'working_mode', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM
# print(f"Before meb_dgm to oni circuits {len(oni_circuits)}")
oni_circuits = oni_circuits.merge(data['MEB_DGM'], left_on='id_dmc', right_on='id', how='left')
# print(f"After meb_dgm to oni circuits {len(oni_circuits)}")

In [50]:
# Usunięcie nulli z oni circuits 'dmc'
oni_circuits = oni_circuits.dropna(subset=['dmc'])

In [51]:
oni_circuits.drop(columns=['id_dmc_y'], inplace=True)
oni_circuits.rename(columns={'id_dmc_x': 'id_dmc'}, inplace=True)

In [ ]:
print(list(oni_circuits.keys()))

In [52]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS
# print(f"Before oni circuits to meb_dmc {len(final_table)}")
final_table = final_table.merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left', suffixes=('_DMC', '_ONI'))
#data['MEB_DMC'].head(1)
# print(f"after oni circuits to meb_dmc {len(final_table)}")

In [ ]:
duplicate_count = final_table['dmc_casting'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc_casting' value: {duplicate_count}")

In [ ]:
# nones_meb_dgm = oni_circuits['dmc'].isna().sum()
# print(nones_meb_dgm)
# oni_circuits = oni_circuits.dropna(subset=['dmc'])
# nones_meb_dgm = oni_circuits['dmc'].isna().sum()
# print(nones_meb_dgm)

In [ ]:
nones_meb_dmc = data['MEB_DMC']['dmc_casting'].isna().sum()
print(nones_meb_dmc)

In [ ]:
nones = final_table['dmc_casting'].isna().sum()
print(nones)

In [ ]:
duplicates = final_table[final_table['dmc_casting'].duplicated(keep=False)]
print("Duplicated rows:")
print(duplicates.head(10))

In [ ]:
print(list(final_table.keys()))

In [ ]:
duplicate_count_oni = final_table['dmc_DMC'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")

In [53]:
final_table.drop(columns=['nok_strefa_ONI', 'nok_rodzaj_ONI', 'status_ko_ONI', 'kod_pola_ONI', 'rodzaj_uszkodzenia_ONI'], inplace=True)
final_table.rename(columns={'nok_strefa_DMC': 'nok_strefa', 'nok_rodzaj_DMC': 'nok_rodzaj', 
                            'status_ko_DMC': 'status_ko', 'kod_pola_DMC': 'kod_pola', 
                            'rodzaj_uszkodzenia_DMC': 'rodzaj_uszkodzenia'}, inplace=True)

In [33]:
# 12.09 - usuwanie anomalii 
# jakieś dziwne rzeczy tu powychodziły
final_table.drop(index=final_table[(final_table['dmc_casting'].duplicated(keep=False)) & (~final_table['dmc_ONI'].isna())].index, inplace=True)
# usuwanie NULL z dmc_casting
final_table.drop(index=final_table[(final_table['dmc_DMC'].duplicated(keep=False)) & (final_table['dmc_casting'].isna())].index, inplace=True)

In [ ]:
final_table

<h1>Usuwanie anomalii</h1>

In [ ]:
print(list(data.keys()))


In [26]:
meb_dgm_df = data['MEB_DGM']

In [ ]:
print(meb_dgm_df.head(5))

In [ ]:
print(list(meb_dgm_df.keys()))

In [ ]:
print(meb_dgm_df['nr_dgm'].unique())
print(len(meb_dgm_df))
print(type(meb_dgm_df['nr_dgm'][0]))
print(len(meb_dgm_df[meb_dgm_df['nr_dgm'] == 20]))

In [ ]:
print(final_table.head(4))

In [ ]:
for i in final_table.keys():
    print(i)

In [ ]:
result = meb_dgm_df.groupby(final_table['dmc_DMC'].str[:3]).size().reset_index(name='count_of_results')
result.columns = ['unique_starting_letters', 'count_of_results']
result = result.sort_values(by='unique_starting_letters')

print(result)

In [ ]:
# wybieranie rekordów dla MEB+ 
meb_dgm_df[(meb_dgm_df.nr_dgm.between(8, 10)) & (meb_dgm_df.dmc.apply(lambda x: len(x)) == 21)]

In [79]:
final_table[final_table.dmc_DMC.str[:3] != '0MH']

21

In [ ]:
pd.set_option('display.max_columns', None)
final_table[(~final_table.status_koncowy.between(1,2)) & (~final_table.dmc_casting.isnull()) & (~final_table.dmc_ONI.isnull())]

In [ ]:
final_table[final_table.status_koncowy.isna()]

In [ ]:
#pd.set_option('display.max_columns', None)
final_table[final_table.dmc_casting.isna()]

In [ ]:
final_table[final_table.dmc_DMC.str[:3] == '0MH']

In [ ]:
final_table[(final_table.id_dmc_ONI.isna()) & (~final_table.status_koncowy.between(1,2))]

In [ ]:
#pd.set_option('display.max_rows', 10)
final_table[(final_table['dmc_casting'].duplicated(keep=False)) & (~final_table.dmc_casting.isna())]

In [ ]:
#pd.set_option('display.max_rows', None)
final_table[(final_table['dmc_casting'].duplicated(keep=False)) & (final_table['dmc_casting'].isna())].sort_values('dmc_DMC')

In [ ]:
final_table[(final_table['dmc_casting'].duplicated(keep=False)) & (~final_table.dmc_ONI.isna())]

In [55]:
# też do wyrzucenia!!!!
final_table[(final_table['dmc_casting'].duplicated(keep=False)) & (~final_table.dmc_ONI.isna())]

,id_dmc_DMC,dmc_DMC,status_koncowy,dmc_casting,nok_strefa,nok_rodzaj,status_ko,kod_pola,rodzaj_uszkodzenia,part_type,...,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,rodzaj_kontroli,product_id,line_id
963776,1009843,0MH301103DVW42305040678413303S222305031429,1.0,230503142930922005780,0.0,0.0,1.0,Brak,Brak,1.0,...,5546.441,71.41204,71.52778,76.15741,152.1412,211.1111,88.0,NaN,NaN,NaN
987933,1010047,0MH301103DVW42305040699413303S222305031429,1.0,230503142930922005780,0.0,0.0,1.0,Brak,Brak,1.0,...,5546.441,71.41204,71.52778,76.15741,152.1412,211.1111,88.0,NaN,NaN,NaN
1018435,1005380,0MH301103DVW42305020231711303S622304292011,1.0,230429201111062092973,0.0,0.0,1.0,Brak,Brak,1.0,...,5017.361,91.60880,64.46759,75.86806,614.7570,363.1944,84.0,NaN,NaN,NaN
1022618,1005383,0MH301103DVW42305020232211303S622304292011,1.0,230429201111062092973,0.0,0.0,1.0,Brak,Brak,1.0,...,5017.361,91.60880,64.46759,75.86806,614.7570,363.1944,84.0,NaN,NaN,NaN
1051730,1109466,0MH301103DVW42307040600624303S222306292006,1.0,230629200610922040436,0.0,0.0,1.0,Brak,Brak,1.0,...,5040.799,86.11111,86.80556,89.46759,345.9491,359.2014,98.0,NaN,NaN,NaN
1056866,1117794,0MH301103DVW42307081428611303S222307080631,1.0,230708063110922046429,NaN,NaN,NaN,NaN,NaN,1.0,...,5145.399,93.86574,90.27778,97.10648,367.7662,387.3264,88.0,NaN,NaN,NaN
1067328,1077610,0MH301103DVW42306127436011303S522306121942,1.0,230612194231052118554,0.0,0.0,1.0,Brak,Brak,1.0,...,4937.500,120.83330,103.87730,100.63660,603.5301,410.5324,85.0,NaN,NaN,NaN
1077615,1106324,0MH301103DVW42307010290111203S222306301946,1.0,230630194610922041355,0.0,0.0,1.0,Brak,Brak,1.0,...,5196.614,90.39352,89.69907,94.90741,327.1991,322.3380,88.0,NaN,NaN,NaN
1077617,1106326,0MH301103DVW42307010290511203S222306301946,1.0,230630194610922041355,0.0,0.0,1.0,Brak,Brak,1.0,...,5196.614,90.39352,89.69907,94.90741,327.1991,322.3380,88.0,NaN,NaN,NaN
1077626,1106358,0MH301103DVW42307010293422203S222307010159,1.0,230701015930922041566,0.0,0.0,1.0,Brak,Brak,1.0,...,5102.431,91.78241,89.93056,95.83333,337.0370,329.6296,88.0,NaN,NaN,NaN


In [56]:
# dmc_casting == NULL - do wyrzucenia!!!!!!!!!
final_table[(final_table['dmc_DMC'].duplicated(keep=False)) & (final_table['dmc_casting'].isna())]

,id_dmc_DMC,dmc_DMC,status_koncowy,dmc_casting,nok_strefa,nok_rodzaj,status_ko,kod_pola,rodzaj_uszkodzenia,part_type,...,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,rodzaj_kontroli,product_id,line_id
1,2064,0MH301103CVW41910090436411102S111910040410,1.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3947,6189,0MH301103CVW41910090436411102S111910040410,1.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8322,12968,0MH301103CVW42001311748711302S112001311512,1.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48044,53002,0MH301103CVW42002061916611102S112002051506,1.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48045,53023,0MH301103CVW42002061916921102S112002051458,1.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121042,1136860,0MH301103DVW42307213311022103S522307210317,0.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1121044,1136862,0MH301103DVW42307213310714103S522307210242,0.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1121047,1136865,0MH301103DVW42307213311213103S122307210322,0.0,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1121918,1136891,0MH301103DVW42307213310824103S522307210325,NaN,None,0.0,0.0,1.0,Brak,Brak,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
final_table[(final_table['dmc_DMC'].duplicated(keep=False)) & (final_table['dmc_casting'].isna())].pivot_table(index='dmc_DMC', columns='status_koncowy')

TypeError: Could not convert BrakBrak to numeric

In [ ]:
final_table[(final_table['dmc_DMC'].duplicated(keep=False)) & (~final_table.dmc_casting.isna())].sort_values('dmc_casting')

In [ ]:
#pd.set_option('display.max_columns', None)
final_table[final_table.dmc_ONI.isna()]

In [ ]:
final_table[final_table.dmc_casting == final_table.dmc_ONI]